<a href="https://colab.research.google.com/github/keiriina/mangroves-chatbot/blob/main/chatbot_with_cohere_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mangrove Chatbot

> Hi! This is a chatbot that answers basic questions about mangroves.


We'll do the following steps:
- **Step 1: Quickstart** - Learn the quickest and easiest way to call the Chat endpoint.
- **Step 2: Creating a custom system prompt** - Steer a chatbot's response toward certain styles, personas, or other characteristics.
- **Step 3: Streaming the response** - Display a chatbot's response incrementally as it is generated, as opposed to waiting for the entire response to be completed.
- **Step 4: Building the Chat History** - Explore how to get the chatbot to maintain the context of the conversation.
- **Step 5: Retrieval-Augmented Generation (RAG): Adding a knowledge base** - Contextualize responses by adding a source of truth where the chatbot should find answers

In [1]:
# install
! pip install cohere python-dotenv requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.3/295.3 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 11.7 MB/s eta 0:00:00


In [8]:
# import
import cohere
import os
import dotenv
import requests

In [3]:
# load env
dotenv.load_dotenv()

co = cohere.ClientV2(os.getenv("COHERE_API_KEY"))

## Define the System Prompt

To get started with Chat, we add a user message to the `messages` parameter and define the model type in the `model` parameter. We then call the Chat endpoint through the client we created earlier.

The response contains several objects. For simplicity, what we want right now is the `text` object.

Here's an example of the assistant responding to a new hire's query asking for help to make introductions.

In [6]:
# Add the user message
message = "I'm teaching about Mangroves in the Philippines. Can you tell me more about mangroves?"

# Create a custom system message
system_message="""## Task and Context
You are a speaker for the locals of a community that needs to conserve their mangrove systems.

## Style Guide
Speak in a way that can be easily understood by anyone, even people who are not expert in English. Be precise."""

# Add the messages
messages = [
    {
        'role': 'system',
        'content': system_message
    },
    {
        'role': 'user',
        'content': message
    }
]

# Generate the response by streaming it
response = co.chat_stream(model="command-r-plus-08-2024",
                          messages=[
                              {
                                  'role':'user',
                                  'content': message
                              }
                          ])

for event in response:
    if event.type == "content-delta":
        print(event.delta.message.content.text, end="")

Mangroves are unique and highly specialized coastal ecosystems found in tropical and subtropical regions. Here's some information about mangroves, specifically focusing on the Philippines:

**Definition and Characteristics:**
- Mangroves refer to both the diverse plant species adapted to brackish, saline coastal habitats and the overall ecosystem they form. These plants have adapted to survive in conditions of low oxygen, high salt concentrations, and periodic tidal flooding.

**Distribution and Diversity in the Philippines:**
- The Philippines is home to a significant proportion of the world's mangrove forests, with over 400,000 hectares of mangroves recorded in the early 2000s. However, this represents a substantial decline from historical estimates, indicating widespread loss of mangrove habitats.
- The country's mangroves are predominantly located along the coasts of Luzon, Palawan, and Mindanao, with the highest concentration in the Western Mindanao-Western Visayas region.
- The P

## Build Knowledge Base

In [7]:
# custom system msg
system_message = """## Task and Context
You are a speaker for the locals of a community that needs to conserve their mangrove systems.

## Style Guide
Speak in a way that can be easily understood by anyone, even people who are not expert in English. Be precise.
"""

# initialize conversation
messages = [
    {
        'role': 'system',
        'content': system_message
    }
]

while True:
    # check if limit true
    user_count = sum(1 for m in messages if m['role'] == 'user')
    if user_count >= 10:
        print("Conversation limit reached (10 exchanges). Please start a new chat.")
        break

    # user input
    user_message = input("You: ")
    if user_message.lower() in ["q"]:
        break

    # append message to convo history
    messages.append({'role': 'user', 'content': user_message})

    # chat stream
    bot_reply = ""
    response = co.chat_stream(
        model="command-r-plus-08-2024",
        messages=messages  # send full history
    )

    for event in response:
        if event.type == "content-delta":
            chunk = event.delta.message.content.text
            print(chunk, end="")
            bot_reply += chunk

    print()

    # bot's reply add to convo history
    messages.append({'role': 'assistant', 'content': bot_reply})


You: i want to teach mangroves, tell me more about it
Mangroves are a group of amazing trees and shrubs that live in a very unique and special habitat. They grow in coastal areas where the land meets the sea, often along muddy shores, estuaries, and tidal creeks. What's really cool about mangroves is that they have special adaptations that allow them to survive in salty water and low-oxygen soil, which is something most plants can't do!

These trees have a very important job. They create a protective barrier along the coast, acting like a shield against strong waves, storms, and even tsunamis. Their roots are like a net that catches sediment and slows down the water, which helps prevent erosion and keeps the soil in place. This also makes the water clearer, which is good for fish and other sea creatures.

Here are some fascinating facts about mangroves:
- Mangrove forests are home to many different animals, including birds, fish, crabs, and even some reptiles like crocodiles!
- They ar